In [3]:
import torch

embedding = torch.rand(10, 56,100,100)

In [4]:
# let's get the client and server model from the embedding
"""
# can't be debugged, move the block below

from Models.mobilenetv2 import MobileNetV2
from Dataloaders.dataloader_cifar10 import Dataloader_cifar10
import torch.nn as nn
import torch.optim as optim
import torch
from tqdm import tqdm
import os
import sys
import cv2
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary

model = MobileNetV2(num_classes=10, weight_path = '/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/MobileNetV2.pth')
# model.load_state_dict(torch.load('/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/MobileNetV2.pth'))
 
# load weights
client_model = model.get_client_model()
server_model = model.get_server_model()

# split the server model into 2 parts
server_model_1 = server_model[:len(server_model)-1]
server_model_2 = server_model[-1]


# put them in the nn.DataParallel
client_model = torch.nn.DataParallel(client_model)
server_model_1 = torch.nn.DataParallel(server_model_1)
server_model_2 = torch.nn.DataParallel(server_model_2)

# client model and server 1 are fixed, server 2 is trainable
client_model = client_model.train()
server_model_1 = server_model_1.train()
server_model_2 = server_model_2.train()

loss = torch.nn.CrossEntropyLoss()
# https://github.com/d-li14/mobilenetv2.pytorch
# batch size 256; epoch 150; learning rate 0.05; LR decay strategy cosine; weight decay 0.00004
optimizer = torch.optim.Adam(server_model_2.parameters(), lr=0.05)

# put them in cuda
client_model = client_model.cuda()
server_model_1 = server_model_1.cuda()
server_model_2 = server_model_2.cuda()

# get the data
train, test, labels = Dataloader_cifar10(train_batch=128, test_batch=100, seed=2014)

# also load the original model
# import F
import torch.nn.functional as F
class MobileNetV22(nn.Module):
    # (expansion, out_planes, num_blocks, stride)
    cfg = [(1,  16, 1, 1),
           (6,  24, 2, 1),  # NOTE: change stride 2 -> 1 for CIFAR10
           (6,  32, 3, 2),
           (6,  64, 4, 2),
           (6,  96, 3, 1),
           (6, 160, 3, 2),
           (6, 320, 1, 1)]

    def __init__(self, num_classes=10):
        super(MobileNetV22, self).__init__()
        # NOTE: change conv1 stride 2 -> 1 for CIFAR10
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layers = self._make_layers(in_planes=32)
        self.conv2 = nn.Conv2d(320, 1280, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(1280)
        self.linear = nn.Linear(1280, num_classes)

    def _make_layers(self, in_planes):
        layers = []
        for expansion, out_planes, num_blocks, stride in self.cfg:
            strides = [stride] + [1]*(num_blocks-1)
            for stride in strides:
                layers.append(Block(in_planes, out_planes, expansion, stride))
                in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.relu(self.bn2(self.conv2(out)))
        # NOTE: change pooling kernel_size 7 -> 4 for CIFAR10
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
class Block(nn.Module):
    '''expand + depthwise + pointwise'''
    def __init__(self, in_planes, out_planes, expansion, stride):
        super(Block, self).__init__()
        self.stride = stride

        planes = expansion * in_planes
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, groups=planes, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_planes)

        self.shortcut = nn.Sequential()
        if stride == 1 and in_planes != out_planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = out + self.shortcut(x) if self.stride==1 else out
        return out
original_model = MobileNetV22()
original_model = original_model.cuda()
print(summary(original_model, (3, 32, 32)))
original_model = original_model.cuda()
original_model = torch.nn.DataParallel(original_model)
original_model.load_state_dict(torch.load('/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/MobileNetV2.pth'))
# train
# epoch = 20
# for i in tqdm(range(epoch)):
#     for data in train:
#         inputs, labels = data
#         inputs = inputs.cuda()
#         labels = labels.cuda()
#         optimizer.zero_grad()
#         # with torch.no_grad():
#         outputs = client_model(inputs)
#         outputs = server_model_1(outputs)
#         outputs = server_model_2(outputs)
#         # get the index of the max value of the output?
#         loss_train = loss(outputs, labels)
#         loss_train.backward()
#         optimizer.step()
#     print('train loss: ',loss_train.item())

# test
correct = 0
total = 0
with torch.no_grad():
    for data in test:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = client_model(inputs)
        outputs = server_model_1(outputs)
        outputs = server_model_2(outputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # the original model
        outputs2 = original_model(inputs)
        _, predicted2 = torch.max(outputs2.data, 1)

# how did I get the labels?
print('outputs ', outputs)
print('predicted ', predicted)
print('outputs2 ', outputs2)
print('predicted2 ', predicted2)
print('labels ', labels)
print('Accuracy of the network on the 10000 test images: %d %%' % ( 100 * correct / total)) 


"""

"\n# can't be debugged, move the block below\n\nfrom Models.mobilenetv2 import MobileNetV2\nfrom Dataloaders.dataloader_cifar10 import Dataloader_cifar10\nimport torch.nn as nn\nimport torch.optim as optim\nimport torch\nfrom tqdm import tqdm\nimport os\nimport sys\nimport cv2\nimport numpy as np\nimport torchvision\nfrom torchvision import datasets, models, transforms\nfrom torchsummary import summary\n\nmodel = MobileNetV2(num_classes=10, weight_path = '/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/MobileNetV2.pth')\n# model.load_state_dict(torch.load('/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/MobileNetV2.pth'))\n \n# load weights\nclient_model = model.get_client_model()\nserver_model = model.get_server_model()\n\n# split the server model into 2 parts\nserver_model_1 = server_model[:len(server_model)-1]\nserver_model_2 = server_model[-1]\n\n\n# put them in the nn.DataParallel\nclient_model = torch.nn.DataParallel(client_model)\nserver_mod

In [5]:
""" finished
import importlib
import 
from Models.mobilenetv2 import MobileNetV2
from Dataloaders.dataloader_cifar10 import Dataloader_cifar10
importlib.reload(MobileNetV2)
importlib.reload(Dataloader_cifar10)
model = MobileNetV2(num_classes=10, weight_path='./otherwork/pytorch-cifar/savedpth/cifar10-mobV2-200.pth')
"""

" finished\nimport importlib\nimport \nfrom Models.mobilenetv2 import MobileNetV2\nfrom Dataloaders.dataloader_cifar10 import Dataloader_cifar10\nimportlib.reload(MobileNetV2)\nimportlib.reload(Dataloader_cifar10)\nmodel = MobileNetV2(num_classes=10, weight_path='./otherwork/pytorch-cifar/savedpth/cifar10-mobV2-200.pth')\n"

In [6]:
"""
# test for loading weights
from otherwork.pytorchcifar.models.mobilenetv2 import MobileNetV2
model = MobileNetV2()
model.load_state_dict(torch.load('./otherwork/pytorchcifar/checkpoint/MobileNetV2.pth')['net'])
"""

"\n# test for loading weights\nfrom otherwork.pytorchcifar.models.mobilenetv2 import MobileNetV2\nmodel = MobileNetV2()\nmodel.load_state_dict(torch.load('./otherwork/pytorchcifar/checkpoint/MobileNetV2.pth')['net'])\n"

In [2]:
""" 
previous test model
# import the layer and test the accuracy

# data 
import Dataloaders.dataloader_cifar10 as Dataloader_cifar10
import Models.mobilenetv2
import otherwork.pytorchcifar.models.mobilenetv2
import torch
train, test, labels = Dataloader_cifar10.Dataloader_cifar10(train_batch=128, test_batch=100, seed=2014)

# model1, original model
model1 = Models.mobilenetv2.MobileNetV2()
client_model, server_model = Models.mobilenetv2.model_splitter(
    weight_path='./otherwork/pytorchcifar/checkpoint/MobileNetV2.pth'
)

# model1 load the weights
model1.load_state_dict(torch.load('./otherwork/pytorchcifar/checkpoint/MobileNetV2.pth')['net'])

# move them to cuda
client_model = client_model.cuda()
server_model = server_model.cuda()
model1 = model1.cuda()

# use first 10 images to test the accuracy
for data in test:
    inputs, labels = data
    inputs = inputs.cuda()
    labels = labels.cuda()
    output2 = model1(inputs)
    outputs = client_model(inputs)
    outputs = server_model(outputs)
    _, predicted = torch.max(outputs.data, 1)
    _, predicted2 = torch.max(output2.data, 1)
    print('predicted ', predicted)
    print('labels ', labels)
    print('predicted2 ', predicted2)
    break
    """

Files already downloaded and verified
Files already downloaded and verified
torch.Size([100, 3, 32, 32])
torch.Size([100, 32, 32, 32])
torch.Size([100, 32, 32, 32])
torch.Size([100, 32, 32, 32])
torch.Size([100, 320, 4, 4])
torch.Size([100, 1280, 4, 4])
torch.Size([100, 1280, 4, 4])
torch.Size([100, 1280, 4, 4])
torch.Size([100, 1280, 1, 1])
torch.Size([100, 1280])


RuntimeError: Given groups=1, weight of size [16, 32, 1, 1], expected input[100, 16, 32, 32] to have 32 channels, but got 16 channels instead

In [ ]:
"""
# test of dimension chagne
import torch
# create a 3*4*5 tensor
array = torch.arange(60).reshape(3,4,5)
print('1-2-3:(1)*4*5 + (2)*5 + (3)', array[1,2,3])
# reshape, 5,4,3
print('change everything from i,j,k to k,i,j, 3-4-5>5-3-4')
print('like flatten the array and change it a new shape. ')
new_array = array.reshape((5,3,4))
print('reshape:')
print('1-2-3:(1)*3*4 + (2)*4 + (3)', new_array[1,2,3])
print('33:(2)*12 + (2)*4 + (1)', new_array[2,2,1])

new_array = array.view((5,3,4))
print('view:')
print('1-2-3:(1)*3*4 + (2)*4 + (3)', new_array[1,2,3])
print('33:(2)*12 + (2)*4 + (1)', new_array[2,2,1])

new_array = array.permute(2,0,1)
print('permute:i,j,k> k,i,j 1,2,3>3,1,2')
print('1-2-3:(1) + (2)*4*5 + (3)*5', new_array[1,2,3])
print('33:(3) + (1)*4*5 + (2)*5', new_array[3,1,2])

new_array = torch.transpose(array,0,2)
new_array = torch.transpose(new_array,1,2)
print('transpose: change only 2 dimensions')
print('1-2-3:(1) + (2)*4*5 + (3)*5', new_array[1,2,3])
print('33:(3) + (1)*4*5 + (2)*5', new_array[3,1,2])

new_array = torch.squeeze(array, (0,1))
print(new_array.shape)
"""

1-2-3:(1)*4*5 + (2)*5 + (3) tensor(33)
change everything from i,j,k to k,i,j, 3-4-5>5-3-4
like flatten the array and change it a new shape. 
reshape:
1-2-3:(1)*3*4 + (2)*4 + (3) tensor(23)
33:(2)*12 + (2)*4 + (1) tensor(33)
view:
1-2-3:(1)*3*4 + (2)*4 + (3) tensor(23)
33:(2)*12 + (2)*4 + (1) tensor(33)
permute:i,j,k> k,i,j 1,2,3>3,1,2
1-2-3:(1) + (2)*4*5 + (3)*5 tensor(56)
33:(3) + (1)*4*5 + (2)*5 tensor(33)
transpose: change only 2 dimensions
1-2-3:(1) + (2)*4*5 + (3)*5 tensor(56)
33:(3) + (1)*4*5 + (2)*5 tensor(33)
torch.Size([3, 4, 5])


In [70]:
"""
# design and test of flatten function
def flatten_model(initial_model, new_model, root_name = ''):
    # recurrsively flatten the model
    # give a model, return two flatten models
    # something like shortcut is static (not dynamic? can be changed dynamically?)
    # this is after loading weights, so we don't need to worry about the weight keys
    
    for sub_name, sub_module in initial_model.named_children():
        if len(list(sub_module.children())) == 0:
            new_model.add_module(root_name + sub_name, sub_module)
        else:
            # if the previous layer a.b.c, now it is a_b_c
            new_model = flatten_model(sub_module, new_model, root_name + sub_name + '_')
    return new_model
flatten_model(model, nn.Sequential())
"""

Sequential(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers_0_conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (layers_0_bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers_0_conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
  (layers_0_bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers_0_conv3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (layers_0_bn3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers_0_shortcut_0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (layers_0_shortcut_1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers_1_conv1): Conv2d(16, 96, kernel_size=(1, 1